In [1]:
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [2]:

import NeuralNetwork.pretrained_models as pre_models
from NeuralNetwork.tools import predict, predict_mul
from NeuralNetwork.preprocessing import skewness_pure, min_max_normalization_pure


def predict_df(X_df):

    model_A2_017, input_features_A2_017 = pre_models.Input_A2_017('weights/')
    
    outputs = ['Input_A2_017']

    preprocessings = ['min_max_normalization']

    models = [model_A2_017]

    input_features = [input_features_A2_017]

    result = {}

    for i in range(len(outputs)):

            if preprocessings[i] == 'skewness':
                X = skewness_pure(X_df, input_features[i])
            elif preprocessings[i] == 'min_max_normalization':
                X = min_max_normalization_pure(X_df, input_features[i])

            result[outputs[i]] = predict_mul(models[i], X)
    
    result_df = pd.DataFrame(result, columns = outputs)
    return result_df


In [3]:
def nn_ave(x, y):

    rmse = []
    r2 = []
    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(
            x, y, test_size=0.2, random_state=i)

        nn_pred = predict_df(X_test)

        rmse.append(math.sqrt(mean_squared_error(y_test, nn_pred)))
        r2.append(r2_score(y_test, nn_pred))

    test_rmse = np.array(rmse).mean()
    print('test_rmse_ave:', test_rmse)
    print(rmse)
    print('\n')
    test_r2 = np.array(r2).mean()
    print('test_r2_ave:', test_r2)
    print(r2)

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [5]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [6]:
fea_number=5
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A2_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
nn=data.drop(['Predict'], axis=1)
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A2_018', 'Input_A2_019', 'Input_A1_019', 'Input_A5_016',
       'Input_A6_016'], dtype=object)

In [7]:
test=pd.read_csv('./final_testX.csv',index_col=0).drop(['Number'], axis=1)[title] ## 改路徑

In [8]:
nn_ave(nn, y)

test_rmse_ave: 0.009059597792077129
[0.009665548366707495, 0.009315590474555059, 0.009226198692886016, 0.009615585673579442, 0.008117916347708877, 0.00801494563233244, 0.00895935363062527, 0.009500887420535686, 0.008678624063709426, 0.009501327618131574]


test_r2_ave: 0.5742793225799125
[0.5843120832560376, 0.572120282248876, 0.5856988589363812, 0.5177480043442706, 0.6153106989835608, 0.6606263182841825, 0.5670983765115472, 0.5179895368734886, 0.60611659385082, 0.5157724725099604]


## Linear Regression Baseline

In [4]:
lr_rmse_ave_fea(skew_data,5)

train_rmse: 0.008300276103545664
test_rmse: 0.009091755270109353
test_r2: 0.5710755789644553


## Lasso

In [5]:
laso_rmse_ave(skew_data[title],y,0.0005)

test_rmse_ave: 0.00910124053474811
[0.009709733726552277, 0.00903254648332744, 0.009161053196929878, 0.009695957276419859, 0.008002412294902086, 0.008361853381021645, 0.009260446777294633, 0.009609651649654691, 0.008750687838157823, 0.009428062723220764]


test_r2_ave: 0.5703366009778013
[0.580502821343885, 0.5977265827157261, 0.5915289017398306, 0.5096525327379231, 0.6261797613027048, 0.6306125850546909, 0.5375127791499521, 0.5068904526864819, 0.5995481395999065, 0.5232114534469117]


## ElasticNet

In [6]:
ElasticNet_rmse_ave(skew_data[title],y,[0.04],0.02)

test_rmse_ave: 0.009176759491504463
[0.009890539087653034, 0.009058458204247498, 0.009167258653892296, 0.009821522348946033, 0.008012159458043138, 0.008452595822201415, 0.009326859337512618, 0.00968870684902373, 0.008904617389020276, 0.009444877764504582]


test_r2_ave: 0.5632259881935459
[0.5647344142441564, 0.595415263829341, 0.5909753391193253, 0.49687005230578496, 0.625268559586482, 0.6225519343413466, 0.5308554116567579, 0.4987438050981534, 0.5853358789188333, 0.5215092228352775]


## SVR

In [7]:
model=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,5,model)

train_rmse: 0.008430247727480519
test_rmse: 0.009470809390685942
test_r2: 0.5339600703664058


## XGB

In [9]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.09,
              booster = 'gbtree', 
              n_estimators = 85, 
              max_depth = 4, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.55,
              reg_alpha =  0,
              reg_lambda = 0.7)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.008837065003264812
[0.009249575666597083, 0.009080898032367389, 0.009076309777001666, 0.009747257208273572, 0.007739830988184788, 0.007863031247109086, 0.00936565923982324, 0.008933484954183593, 0.008048113055456553, 0.009266489863651159]


test_r2_ave: 0.5941379546490498
[0.6193217862545504, 0.5934082871763435, 0.5990509921904466, 0.5044500888430413, 0.65030943168478, 0.6733693001207957, 0.5269439929358045, 0.573842690147869, 0.6612696823484336, 0.5394132947884347]


## RandomForest

In [10]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 90,
            max_depth = 7,
            min_samples_split = 9,
            max_leaf_nodes = 11,
            min_samples_leaf =3,
            random_state = 42) 


In [11]:
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.008925843357390223
[0.009210125256724678, 0.00913248179150185, 0.00920898732491742, 0.009937240180769243, 0.008075117896793299, 0.007625395239402569, 0.009415627147691322, 0.00915181698445304, 0.008390263020983843, 0.009111378730664966]


test_r2_ave: 0.5856896471260756
[0.6225621262697759, 0.5887759042016061, 0.5872431652137791, 0.48494438801992423, 0.6193562459621168, 0.6928137895868074, 0.5218828062346494, 0.5527578001899157, 0.6318565454027938, 0.5547037001793874]


In [9]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=0.0005)
elastic_mod=ElasticNet(alpha=[0.04], l1_ratio=0.02)
svr_mod=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.09,
              booster = 'gbtree', 
              n_estimators = 85, 
              max_depth = 4, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.55,
              reg_alpha =  0,
              reg_lambda = 0.7)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 90,
            max_depth = 7,
            min_samples_split = 9,
            max_leaf_nodes = 11,
            min_samples_leaf =3,
            random_state = 42) 
# vote_mod = VotingRegressor([ ('Lasso', lasso_mod),('SVR', svr_mod), ('Elastic', elastic_mod), 
#                             ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])
vote_mod = VotingRegressor([ ('Lasso', lasso_mod),('Linear', lr), ('Elastic', elastic_mod), 
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])

voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.00889863447687027
[0.009427251530600493, 0.008953520780936738, 0.008984486252814462, 0.009615148067597295, 0.007859793921422806, 0.008023409471035517, 0.009133327424095918, 0.00929199756413655, 0.008473821854271079, 0.00922358790179184]


test_r2_ave: 0.5890730559732271
[0.6045563616482452, 0.6047347668208524, 0.6071226244594108, 0.5177918979901129, 0.6393854189802288, 0.6599091778634315, 0.5501228748437128, 0.5389518367175089, 0.624487332919456, 0.5436682674893123]


In [10]:

stack_mod = StackingRegressor(regressors=[lasso_mod, elastic_mod,  vote_mod], 
                           meta_regressor=random_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.009061637706291526
[0.009320568464478644, 0.009014933038635545, 0.009077167649572828, 0.009878767431130737, 0.00786205492261383, 0.008117294736967007, 0.010015441329449247, 0.009150438558226939, 0.008427488641106746, 0.009752222290733739]


test_r2_ave: 0.5724157306538572
[0.6134557604285777, 0.5992939175359935, 0.5989751949953306, 0.49098793951818953, 0.639177915506252, 0.6519035219878184, 0.45902644021573336, 0.5528925152856138, 0.6285825670104612, 0.48986153405460153]


In [11]:
ave(skew_data[title],y,0.2,stack_mod,0.5,vote_mod,0.3,xgb_mod)

test_rmse_ave: 0.00883982456524144
[0.009296311118682793, 0.008918413337473322, 0.00889626131537468, 0.00962489740788272, 0.007772220058134946, 0.007946935330656388, 0.009255430961187012, 0.009115031383453522, 0.008340705362820068, 0.00923203937674893]


test_r2_ave: 0.5942029490732004
[0.6154651519836832, 0.607828420426012, 0.6148006146712248, 0.5168135263097005, 0.6473765896845145, 0.6663613492527618, 0.5380136453984807, 0.5563459420202868, 0.6361926303987517, 0.5428316205865871]


In [12]:
name='Input_A2_017'
test_pred=[]
seed=[4,5,2,8,7,9]
stack_w=0.2
vote_w=0.5
model_w=0.3
model=xgb_mod
for i in seed:
    X_train, X_test, y_train, y_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)

    stack = stack_mod.fit(X_train, y_train)
    stack_pred = stack.predict(test)
    vote = vote_mod.fit(X_train, y_train)
    vote_pred = vote.predict(test)
    mod = model.fit(X_train, y_train)
    mod_pred = mod.predict(test)

    ###
    final_test = (vote_w*vote_pred+stack_w*stack_pred + model_w*mod_pred)
    ###
    test_pred.append(final_test)



final_test = pd.DataFrame(np.mean(test_pred, axis=0), columns=[name])
final_test.to_csv(('./output/'+name+'.csv'))